# Homework 2: 기계학습 응용

Author: GPT 자동 생성

---

## 1. D2L Chapter 3
[D2L 책](https://d2l.ai/chapter_linear-regression/index.html)의 선형 회귀 챕터를 읽고 예제 코드를 따라 실행했습니다. 이어서 Ohm's Law 실험을 진행합니다.

## 2. Ohm's Law 실험: 전압-전류 관계 모델링
측정된 전류(`current`)와 전압(`voltage`) 데이터를 기반으로 다음 세 가지 모델을 최소제곱법(Least Mean Squares Regression)으로 학습합니다.

1. **이차 모델**: $V = c + rI + qI^2$
2. **선형 모델**: $V = c + rI$
3. **Ohm's Law (편향 없음)**: $V = rI$

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
# Current and voltage data
current = torch.tensor([1.5420291, 1.8935232, 2.1603365, 2.5381863, 2.893443, 
                    3.838855, 3.925425, 4.2233696, 4.235571, 4.273397, 
                    4.9332876, 6.4704757, 6.517571, 6.87826, 7.0009003, 
                    7.035741, 7.278681, 7.7561755, 9.121138, 9.728281])
voltage = torch.tensor([63.802246, 80.036026, 91.4903, 108.28776, 122.781975, 
                    161.36314, 166.50816, 176.16772, 180.29395, 179.09758, 
                    206.21027, 272.71857, 272.24033, 289.54745, 293.8488, 
                    295.2281, 306.62274, 327.93243, 383.16296, 408.65967])

In [ ]:
# Dictionary for losses and results
losses_dict = {}
results_dict = {}

# Training function definition
def train_model(model_fn, params, name):
    optimizer = optim.LBFGS(params, lr=1.0, max_iter=500)
    losses = []

    def closure():
        optimizer.zero_grad()
        preds = model_fn(current)
        loss = torch.mean((preds - voltage) ** 2)
        loss.backward()
        losses.append(loss.item())
        return loss

    for _ in range(20):
        optimizer.step(closure)

    final_loss = losses[-1]
    final_params = [p.item() for p in params]

    losses_dict[name] = losses
    results_dict[name] = {
        "loss": final_loss,
        "params": final_params,
        "pred": model_fn(current).detach()
    }

    print(f"📌 [{name}] 최종 손실: {final_loss:.4f}")
    print(f"   학습된 파라미터: {final_params}")
    return results_dict[name]["pred"]


In [ ]:
# Model definitions and training

# 이차 모델: V = c + rI + qI^2
c1, r1, q1 = [torch.randn(1, requires_grad=True) for _ in range(3)]
quad_model = lambda x: c1 + r1 * x + q1 * x ** 2
quad_pred = train_model(quad_model, [c1, r1, q1], "이차 모델")

# 선형 모델: V = c + rI
c2, r2 = [torch.randn(1, requires_grad=True) for _ in range(2)]
lin_model = lambda x: c2 + r2 * x
lin_pred = train_model(lin_model, [c2, r2], "선형 모델")

# Ohm의 법칙: V = rI
r3 = torch.randn(1, requires_grad=True)
ohm_model = lambda x: r3 * x
ohm_pred = train_model(ohm_model, [r3], "Ohm의 법칙")

## 3. 보너스 문제: Wien의 근사법을 이용한 Temperature 추정

흑체(black body)의 복사 스펙트럼은 Temperature에 따라 달라지며, Wien의 근사식을 이용하면 주어진 Wavelength에서의 방사 에너지로부터 Temperature를 유추할 수 있다.

Wien의 근사식은 다음과 같다:

\[
B_\lambda(T) = \frac{2 h c^2}{\lambda^5} \exp\left(-\frac{h c}{\lambda k T}\right)
\]

여기서:
- \( B_\lambda \): Wavelength λ에서의 스펙트럼 Spectral Radiance
- \( h \): 플랑크 상수
- \( c \): 빛의 속도
- \( k \): 볼츠만 상수
- \( T \): Temperature

이번 문제에서는 PyTorch를 이용해 주어진 방사 스펙트럼 데이터를 바탕으로 흑체의 Temperature를 추정한다.

In [ ]:
import torch
import matplotlib.pyplot as plt

# Constant definitions
c = 299792458                 # 빛의 속도 (m/s)
h = 6.62607004e-34            # 플랑크 상수 (J·s)
k = 1.38064852e-23            # 볼츠만 상수 (J/K)
lamscale = 1e-6               # μm 단위를 m로 변환하기 위한 스케일

# Wien 근사식 정의
p_out = 2 * h * c**2 / lamscale**5
p_in = (h / k) * (c / lamscale)

def wien(lam, t):
    return (p_out / lam**5) * torch.exp(-p_in / (lam * t))

### 3-1. 측정 데이터 생성
True Temperature가 273K인 흑체를 가정하고, 3μm ~ 19μm의 Wavelength에서 Spectral Radiance를 측정했다고 가정한다. 센서의 오차를 반영하기 위해 노이즈를 추가한다.

In [ ]:
# 실제 온도
realtemp = 273

# 파장 범위 (μm)
wavelengths = torch.arange(3, 20, 2)

# Simulated measurements (노이즈 포함)
delta = torch.randn(len(wavelengths)) * 0.5
radiance = wien(wavelengths + delta, realtemp)

# 측정값 시각화
plt.plot(wavelengths.numpy(), radiance.numpy(), 'ko-', label='측정값')
plt.plot(wavelengths.numpy(), wien(wavelengths, realtemp).detach().numpy(), 'b--', label='실제 (273K)')
plt.xlabel("파장 (μm)")
plt.ylabel("복사 에너지")
plt.title("측정값 vs 실제 스펙트럼")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

### 3-2. Temperature 추정
Wavelength에 따른 Spectral Radiance Measured을 이용해 PyTorch로 Temperature를 추정한다.
초기 Temperature는 200K로 두고, 최적화를 통해 True Temperature에 근접하도록 한다.
Loss 함수는 로그 기반 MSE를 사용한다.

In [ ]:
# Temperature parameter to optimize
T = torch.nn.Parameter(torch.tensor([200.0]))

# Optimizer
optimizer = torch.optim.LBFGS([T], lr=0.3, max_iter=100, line_search_fn='strong_wolfe')

# Log-based loss function
def log_loss(y_pred, y_true):
    return torch.mean((torch.log(y_pred) - torch.log(y_true))**2)

# Optimization loop
losses = []

def closure():
    optimizer.zero_grad()
    pred = wien(wavelengths, T)
    loss = log_loss(pred, radiance)
    loss.backward()
    losses.append(loss.item())
    return loss

for _ in range(15):
    optimizer.step(closure)

print(f"예측된 온도: {T.item():.2f}K / 실제 온도: {realtemp}K")
print(f"최종 손실 값: {losses[-1]:.6f}")

### 3-3. 결과 Visualization

In [ ]:
# Prediction calculation
with torch.no_grad():
    estimated = wien(wavelengths, T)
    true_radiance = wien(wavelengths, realtemp)

# Loss decrease visualization
plt.figure(figsize=(6, 4))
plt.plot(losses, marker='o')
plt.title("손실 감소 추이")
plt.xlabel("반복")
plt.ylabel("로그 손실")
plt.grid(True)
plt.tight_layout()
plt.show()

# Comparison graph of spectral radiance
plt.figure(figsize=(8, 5))
plt.plot(wavelengths.numpy(), radiance.numpy(), 'ko-', label="측정값")
plt.plot(wavelengths.numpy(), estimated.numpy(), 'r--', label=f"추정 (T={T.item():.1f}K)")
plt.plot(wavelengths.numpy(), true_radiance.numpy(), 'b:', label="실제 (273K)")

plt.text(0.03, 0.97, f"예측 T = {T.item():.2f}K\n최종 손실 = {losses[-1]:.4e}",
         transform=plt.gca().transAxes,
         fontsize=12, verticalalignment='top',
         bbox=dict(boxstyle="round", facecolor="white", alpha=0.8))

plt.xlabel("파장 (μm)")
plt.ylabel("복사 에너지")
plt.title("복사 스펙트럼 비교")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()